In [ ]:
#importing required modules

In [3]:
import featuretools as ft
import featuretools.variable_types as vtypes
import pandas as pd
import nltk
import numpy as np
import re

In [ ]:
#importing the dataset

In [4]:
loandata = pd.read_csv("loan_prep.csv")

In [5]:
loandata.head(5)

,id,member_id,loan_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,total_pymnt_inv
0,1077501,1296599,5000.0,4975.0,36 months,10.65,B,B2,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,27.65,0.0,1.0,9.0,5861.071414,5831.78
1,1077430,1314167,2500.0,2500.0,60 months,15.27,C,C4,RENT,30000.0,Source Verified,Charged Off,car,GA,1.00,0.0,5.0,4.0,1008.710000,1008.71
2,1077175,1313524,2400.0,2400.0,36 months,15.96,C,C5,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,8.72,0.0,2.0,10.0,3003.653644,3003.65
3,1076863,1277178,10000.0,10000.0,36 months,13.49,C,C1,RENT,49200.0,Source Verified,Fully Paid,other,CA,20.00,0.0,1.0,37.0,12226.302212,12226.30
4,1075358,1311748,3000.0,3000.0,60 months,12.69,B,B5,RENT,80000.0,Source Verified,Current,other,OR,17.94,0.0,0.0,38.0,3242.170000,3242.17


In [6]:
type(loandata)

pandas.core.frame.DataFrame

In [ ]:
#creating entity sets

In [7]:
es = ft.EntitySet()
es = es.entity_from_dataframe(entity_id="loandata",
                                  dataframe=loandata,
                                  index="id")

In [8]:
es["loandata"].variables

[<Variable: id (dtype = index)>,
 <Variable: member_id (dtype = numeric)>,
 <Variable: loan_amnt (dtype = numeric)>,
 <Variable: funded_amnt_inv (dtype = numeric)>,
 <Variable: term (dtype = categorical)>,
 <Variable: int_rate (dtype = numeric)>,
 <Variable: grade (dtype = categorical)>,
 <Variable: sub_grade (dtype = categorical)>,
 <Variable: home_ownership (dtype = categorical)>,
 <Variable: annual_inc (dtype = numeric)>,
 <Variable: verification_status (dtype = categorical)>,
 <Variable: loan_status (dtype = categorical)>,
 <Variable: purpose (dtype = categorical)>,
 <Variable: addr_state (dtype = categorical)>,
 <Variable: dti (dtype = numeric)>,
 <Variable: delinq_2yrs (dtype = numeric)>,
 <Variable: inq_last_6mths (dtype = numeric)>,
 <Variable: total_acc (dtype = numeric)>,
 <Variable: total_pymnt (dtype = numeric)>,
 <Variable: total_pymnt_inv (dtype = numeric)>]

In [9]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
    target_entity = 'loandata', 
    max_depth = 3, 
    verbose = 3, 
    n_jobs = 1)

Built 19 features
Elapsed: 00:30 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


In [11]:
feature_matrix.head()

,member_id,loan_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,total_pymnt_inv
id,,,,,,,,,,,,,,,,,,,
54734,80364,25000.0,19080.057198,36 months,11.89,B,B4,RENT,85000.0,Verified,Fully Paid,debt_consolidation,CA,19.48,0.0,0.0,42.0,29324.32,21811.70
55521,107577,1000.0,0.000000,36 months,16.08,F,F2,RENT,30000.0,Not Verified,Does not meet the credit policy. Status:Fully ...,debt_consolidation,IL,23.84,0.0,1.0,15.0,1207.76,0.00
55742,114426,7000.0,672.803839,36 months,10.71,B,B5,RENT,65000.0,Not Verified,Fully Paid,credit_card,NY,14.29,0.0,0.0,7.0,8215.45,684.85
56413,129814,7000.0,0.007494,36 months,16.08,F,F2,MORTGAGE,189500.0,Verified,Does not meet the credit policy. Status:Charge...,debt_consolidation,CA,22.47,0.0,4.0,31.0,1231.90,0.00
56705,133361,11000.0,11000.000000,36 months,9.99,B,B3,MORTGAGE,33500.0,Source Verified,Current,debt_consolidation,MO,18.38,0.0,0.0,23.0,376.25,376.25


In [ ]:
#normalizing the dataset

In [12]:
es = es.normalize_entity(base_entity_id='loandata', new_entity_id='loan_amnt', index='loan_amnt')

In [13]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
    target_entity = 'loandata', 
    max_depth = 3, 
    verbose = 3, 
    n_jobs = 1)

Built 96 features
Elapsed: 07:56 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


In [14]:
feature_matrix.columns

Index(['member_id', 'loan_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'total_acc', 'total_pymnt',
       'total_pymnt_inv', 'loan_amnt.SUM(loandata.member_id)',
       'loan_amnt.SUM(loandata.funded_amnt_inv)',
       'loan_amnt.SUM(loandata.int_rate)',
       'loan_amnt.SUM(loandata.annual_inc)', 'loan_amnt.SUM(loandata.dti)',
       'loan_amnt.SUM(loandata.delinq_2yrs)',
       'loan_amnt.SUM(loandata.inq_last_6mths)',
       'loan_amnt.SUM(loandata.total_acc)',
       'loan_amnt.SUM(loandata.total_pymnt)',
       'loan_amnt.SUM(loandata.total_pymnt_inv)',
       'loan_amnt.STD(loandata.member_id)',
       'loan_amnt.STD(loandata.funded_amnt_inv)',
       'loan_amnt.STD(loandata.int_rate)',
       'loan_amnt.STD(loandata.annual_inc)', 'loan_amnt.STD(loandata.dti)',
       'loan_amnt.STD(loandata.delinq_2yrs

In [21]:
# loandata[loandata['loan_amnt']==5000]['funded_amnt_inv'].sum()

135026592.67570522

In [15]:
feature_matrix.to_csv("ft.csv")